## Define Slice Topology

### Imports 

In [ ]:
# Import libraries
%run ../setup/include/include_libraries.py
# load general variables
%run ../setup/config/load_variables.py
# load topology variables
%run topology_variables.ipynb

from state import *
import write_value

### Slice Topology

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # we will use CX5 to generate traffic so need sites that have CX5 for this example.
    cx5_column_name = 'nic_connectx_5_available'

    # find a site with available ConnectX-5 
    sites = [site1] = fablib.get_random_sites(count=1, filter_function=lambda x: x[cx5_column_name] > 0)
    print(f"Sites: {sites}")
    
    if (node1_site ==''):
        node1_site = site1
        write_value.write_value_to_file('topology_variables.ipynb', 'node1_site', node1_site)
    if (node2_site ==''):
        node2_site = site1
        write_value.write_value_to_file('topology_variables.ipynb', 'node2_site', node2_site)
    if (node3_site ==''):
        node3_site = site1
        write_value.write_value_to_file('topology_variables.ipynb', 'node3_site', node3_site)
        
    # add bridge node
    bridge1 = slice.add_node(name=node3_name, site=node3_site, cores=node3_cores, ram=node3_ram, disk=node3_disk, image=default_image)
    bridge1_nic = bridge1.add_component(model=node3_nic_model, name=node3_nic_name)
    
    # add host nodes
    site_node_count = 2
    print(f"Adding nodes to {site1}")
    for node_num in range(site_node_count):
        node_name = globals()[f'node{node_num+1}_name']

        node = slice.add_node(name=node_name, site=globals()[f'node{node_num+1}_site'], cores=globals()[f'node{node_num+1}_cores'], ram=globals()[f'node{node_num+1}_ram'], disk=globals()[f'node{node_num+1}_disk'], image=default_image)
        iface = node.add_component(model=globals()[f'node{node_num+1}_nic_model'], name=globals()[f'node{node_num+1}_nic_name']).get_interfaces()[0]    
        net = slice.add_l2network(name=f"net{node_num}")

        net.add_interface(iface)
        net.add_interface(bridge1_nic.get_interfaces()[i])

    
    success = True
    
except Exception as e:
    print(f"Exception: {e}")
    success = False
    raise SystemExit("Stopping notebook execution due to exception")

### Set State to "SLICE_DEFINED" 

In [ ]:
if success:
    state_data = {'slice_name': slice_name, 'state': ['SLICE_DEFINED']}
    set_state_local(state_data)
else:
    print ('State is not updated due to error')